# MISP

In [1]:
%pip install pymisp


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
from dotenv import dotenv_values

In [4]:
from surianalytics.connectors import getGitRoot, check_str_bool

In [5]:
import os
import shutil

In [6]:
config = dotenv_values(os.path.join(getGitRoot(), ".env") if shutil.which("git") is not None else os.path.join(os.path.expanduser("~"), ".env"))

In [7]:
from pymisp import PyMISP

In [8]:
misp = PyMISP(config["MISP_HOST"],
              config["MISP_TOKEN"],
              check_str_bool(config["MISP_TLS_VERIFY"]),
              debug=False)

In [9]:
publish_timestamp="7d"

In [10]:
import pandas as pd

In [11]:
attributes = [
    "url",
    "domain",
    "ip-dst",
    "ip-dst|port"
]

In [12]:
DF_ATTR = pd.DataFrame()
for attr in attributes:
    resp = pd.json_normalize(misp.search(controller='attributes', 
                                                    publish_timestamp=publish_timestamp, 
                                                    type_attribute=attr, 
                                                    category="Network activity", 
                                                    pythonify=False).get("Attribute"))
    DF_ATTR = pd.concat([DF_ATTR, resp], axis=0)
    print(attr, len(resp))

url 306
domain 501
ip-dst 326
ip-dst|port 6


In [16]:
DF_ATTR.columns.values

array(['id', 'event_id', 'object_id', 'object_relation', 'category',
       'type', 'to_ids', 'uuid', 'timestamp', 'distribution',
       'sharing_group_id', 'comment', 'deleted', 'disable_correlation',
       'first_seen', 'last_seen', 'value', 'Event.org_id',
       'Event.distribution', 'Event.id', 'Event.info', 'Event.orgc_id',
       'Event.uuid', 'Object.id', 'Object.distribution',
       'Object.sharing_group_id'], dtype=object)

In [13]:
DF_ATTR.head(3)

,id,event_id,object_id,object_relation,category,type,to_ids,uuid,timestamp,distribution,...,value,Event.org_id,Event.distribution,Event.id,Event.info,Event.orgc_id,Event.uuid,Object.id,Object.distribution,Object.sharing_group_id
0,319508,1580,20531,url,Network activity,url,True,0219ad8f-579c-4bbf-97c9-582b81c67507,1681200514,5,...,https://rentry.co/shitonyourAV/raw,2,3,1580,Malicious GitHub user and account - distributi...,4,659a6331-0690-4b3b-ae16-e29a1fc31fc2,20531,5,0
1,319509,1580,20532,url,Network activity,url,True,749d9d77-faaf-4834-9342-4a50e98b945b,1681200522,5,...,https://rentry.co/shitbymyself/raw,2,3,1580,Malicious GitHub user and account - distributi...,4,659a6331-0690-4b3b-ae16-e29a1fc31fc2,20532,5,0
2,319510,1580,20533,url,Network activity,url,True,ae69ac2a-e85b-49b3-ac7c-65069043d600,1681200530,5,...,https://rentry.co/9ops5/raw,2,3,1580,Malicious GitHub user and account - distributi...,4,659a6331-0690-4b3b-ae16-e29a1fc31fc2,20533,5,0


In [14]:
DF_ATTR.groupby("type").agg({"value": "nunique",
                             "event_id": "nunique"})

,value,event_id
type,,
domain,493,24
ip-dst,323,27
ip-dst|port,6,1
url,292,26


In [15]:
DF_ATTR.groupby("Event.info").agg({"type": ["unique", "nunique"],
                                   "value": ["unique", "nunique"]})

type  \
                                                                        unique   
Event.info                                                                       
AA23-136A StopRansomware BianLian Ransomware Group                       [url]   
Amazon‑themed campaigns of Lazarus in the Nethe...               [url, ip-dst]   
An Analysis of Infrastructure linked to the Hag...            [domain, ip-dst]   
Bad magic: new APT found in the area of Russo-U...            [domain, ip-dst]   
Blind Eagle Deploys Fake UUE Files and Fsociety...               [url, ip-dst]   
Cloud Atlas targets entities in Russia and Bela...            [domain, ip-dst]   
CloudWizard APT: the bad magic story goes on                  [domain, ip-dst]   
CrowdStrike Falcon Platform Detects and Prevent...               [url, domain]   
DTrack activity targeting Europe and Latin America                    [domain]   
DarkBit Ransomware Targets Israel with Command-...                       [url]   
Disrupting SEABORGIUM’s ongoing phishing operat...                    [domain]   
Dossier Center Investigation: Prigozhin's Cyber...                    [ip-dst]   
Emotet coming in hot                                     [url, domain, ip-dst]   
Gamaredon (Ab)uses Telegram to Target Ukrainian...               [url, ip-dst]   
Gamaredon APT targets Ukrainian government agen...       [url, domain, ip-dst]   
HALFRIG - Malware Analysis Report                                [url, domain]   
IcedID infection from fake Microsoft Teams page          [url, domain, ip-dst]   
In the footsteps of the Fancy Bear: PowerPoint ...                       [url]   
Iran: State-Backed Hacking of Activists, Journa...               [url, domain]   
Is Hagga Threat Actor (ab)using FSociety framew...       [url, domain, ip-dst]   
Malicious GitHub user and account - distributin...                       [url]   
Mustang Panda APT Group Uses European Commissio...                    [ip-dst]   
Mustang Panda Uses the Russian-Ukrainian War to...                    [ip-dst]   
Mustang Panda deploys a new wave of malware tar...  [url, ip-dst, ip-dst|port]   
NOBELIUM Uses Poland's Ambassador’s Visit to th...               [url, ip-dst]   
New MortalKombat ransomware and Laplas Clipper ...       [url, domain, ip-dst]   
NewsPenguin, a Previously Unknown Threat Actor,...               [url, ip-dst]   
No Pineapple! –DPRK Targeting of Medical Resear...                    [ip-dst]   
Prometei botnet improves modules and exhibits n...       [url, domain, ip-dst]   
QUARTERRIG - Malware Analysis Report                     [url, domain, ip-dst]   
SNOWYAMBER - Malware Analysis Report                             [url, domain]   
SNOWYAMBER, HALFRIG, QUARTERRIG - IoC Reference          [url, domain, ip-dst]   
Sweet QuaDreams A First Look at Spyware Vendor ...                    [domain]   
TA505 Group’s TeslaGun In-Depth Analysis                      [domain, ip-dst]   
Talos uncovers espionage campaigns targeting CI...       [url, domain, ip-dst]   
The distinctive rattle of APT SideWinder                 [url, domain, ip-dst]   
Under the hood of a Doppelgänger                              [domain, ip-dst]   
Untangling KNOTWEED: European private-sector of...                    [domain]   
WIP26 Espionage | Threat Actors Abuse Cloud Inf...               [url, ip-dst]   
X_Trader Supply Chain Attack Affects Critical I...                       [url]   

                                                            \
                                                   nunique   
Event.info                                                   
AA23-136A StopRansomware BianLian Ransomware Group       1   
Amazon‑themed campaigns of Lazarus in the Nethe...       2   
An Analysis of Infrastructure linked to the Hag...       2   
Bad magic: new APT found in the area of Russo-U...       2   
Blind Eagle Deploys Fake UUE Files and Fsociety...       2   
Cloud Atlas targets entities in Russia and Bela... 

# Scan for IoC values via scirius

## Domains

## IP addrs